# Testing different parameters

In [1]:
import os
import copy
from matplotlib import pyplot as plt
from cascade.cascade import Cascade
import numpy as np

# ki=4.75

In [2]:
datadir = "C:/Users/Lexi/PycharmProjects/CASCADE/cascade/data/outwash_data/"  # input datadir, add the default b3d files here
cascade2 = Cascade(
    datadir, 
    name="default",  
    elevation_file="NCB-default-elevation-left-damMHW.npy", # input in dam MHW because we do do any conversions to it
    dune_file="NCB-default-dunes-left-dam.npy",  # dam above berm elevation
    parameter_file="outwash-parameters.yaml",
    storm_file="cascade-default-storms.npy",  # same as "StormSeries_1kyrs_VCR_Berm1pt9m_Slope0pt04_01.npy"
    num_cores=1,  # cascade can run in parallel, can never specify more cores than that
    roadway_management_module=False,
    alongshore_transport_module=False,
    beach_nourishment_module=False,
    community_economics_module=False,
    outwash_module=True,
    alongshore_section_count=1,
    time_step_count=2,
    wave_height=1, # ---------- for BRIE and Barrier3D --------------- #
    wave_period=7,
    wave_asymmetry=0.8,
    wave_angle_high_fraction=0.2,
    bay_depth=3.0,
    s_background=0.001,
    berm_elevation=1.46,  # m NAVD88, gets converted to dam MHW in load_inputs, added to dunes
    MHW=0.36,  # used to convert berm elevation from NAVD88 to MHW
    beta=0.03,
    sea_level_rise_rate=0.004,
    sea_level_rise_constant=True,
    background_erosion=0.0,
    min_dune_growth_rate=0.25,
    max_dune_growth_rate=0.65,
    road_ele=1.7,  # ---------- roadway management --------------- #
    road_width=30,
    road_setback=30,
    dune_design_elevation=3.7,
    dune_minimum_elevation=2.2,
    trigger_dune_knockdown=False,
    group_roadway_abandonment=None,
    nourishment_interval=None, # ---------- beach and dune ("community") management --------------- #
    nourishment_volume=300.0,
    overwash_filter=40,
    overwash_to_dune=10,
    number_of_communities=1, # ---------- coastal real estate markets (in development) --------------- #
    sand_cost=10,
    taxratio_oceanfront=1,
    external_housing_market_value_oceanfront=6e5,
    external_housing_market_value_nonoceanfront=4e5,
    fixed_cost_beach_nourishment=2e6,
    fixed_cost_dune_nourishment=2e5,
    nourishment_cost_subsidy=10e6,
    house_footprint_x=15,
    house_footprint_y=20,
    beach_full_cross_shore=70,
    outwash_storms_file="outwash_storms_6min_short.npy",  # --------- outwasher (in development) ------------ #
    # hydrograph in dam
    percent_washout_to_shoreface=100,
    outwash_beach_file = "NCB-default-beach-left-dam.npy",  
    dune_flow_dynamics="full",
    outwasher_substep = 15,
    ki_value=4.75E-3,
    cx=0.65
)

for time_step in range(cascade2._nt - 1):
    # Print time step to screen (NOTE: time_index in each model is time_step+1)
    print("\r", "Time Step: ", time_step, end="")
    cascade2.update()
    if cascade2.b3d_break:
        break

ValidationError: 2 validation errors for Barrier3dConfiguration
elevation_file
  file or directory at path "NCB-default-elevation-left-damMHW-test.npy" does not exist (type=value_error.path.not_exists; path=NCB-default-elevation-left-damMHW-test.npy)
dune_file
  file or directory at path "NCB-default-dunes-left-dam-test.npy" does not exist (type=value_error.path.not_exists; path=NCB-default-dunes-left-dam-test.npy)

In [ ]:
# plt.rcParams['figure.figsize'] = (10,6)
# plt.rcParams.update({"font.size": 15})

initial_domain2 = cascade2.outwash[0]._initial_full_domain
final_domain2 = cascade2.outwash[0]._full_domain
domain_change2 = final_domain2-initial_domain2

### ELEVATION PLOTS-------------------------------------------------------------------------------------------------------------
# plotting initial domain
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
mat = ax1.matshow(
    initial_domain2*10,
    cmap="terrain",
    vmin=-3, vmax=6.0,
)
cbar = fig1.colorbar(mat)
cbar.set_label('m MHW', rotation=270, labelpad=15)
ax1.set_title("Initial Elevation")
ax1.set_ylabel("barrier width (dam)")
ax1.set_xlabel("barrier length (dam)")
plt.gca().xaxis.tick_bottom()
xtick_max = np.shape(initial_domain2)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 5))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(initial_domain2)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 5))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

# plotting post storm elevation
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
mat2 = ax2.matshow(
    final_domain2[:, :]*10,
    cmap="terrain",
    vmin=-3, vmax=6.0,
)
ax2.set_xlabel('barrier length (dam)')
ax2.set_ylabel('barrier width (dam)')
ax2.set_title("Substep = 20\nFinal Elevation")
plt.gca().xaxis.tick_bottom()
cbar = fig2.colorbar(mat2)
cbar.set_label('m MHW', rotation=270, labelpad=15)
xtick_max = np.shape(final_domain2)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 5))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(final_domain2)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 5))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

# plotting domain elevation change
fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
mat3 = ax3.matshow(
    domain_change2*10,
    cmap="seismic",
    vmin=-5, vmax=5,
)
ax3.set_xlabel('barrier length (dam)')
ax3.set_ylabel('barrier width (dam)')
ax3.set_title("Substep = 20\nElevation Change")
plt.gca().xaxis.tick_bottom()
cbar = fig3.colorbar(mat3)
cbar.set_label('meters', rotation=270, labelpad=15)
xtick_max = np.shape(domain_change2)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 5))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(domain_change2)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 5))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)

print("outwash flux is: ", cascade2.outwash[0]._outwash_flux_TS[0], " m3/m")
print("outwash volume is: ", cascade2.outwash[0]._outwash_TS[0], " m3")


In [ ]:
initial = cascade2.outwash[0]._initial_full_domain
final = cascade2.outwash[0]._full_domain
change = final-initial

plt.rcParams['figure.figsize'] = (30,10)
plt.rcParams.update({"font.size": 20})

shrink=0.5

fig1 = plt.figure()
fig1.tight_layout()
fig1.suptitle('Configuration 1: Modeled Erosion', weight="bold")
ax1 = fig1.add_subplot(131)
mat = ax1.matshow(
    initial*10,
    cmap="terrain",
    vmin=-3.0,
    vmax=6.0,
)
cbar = fig1.colorbar(mat, shrink=shrink)
cbar.set_label('m MHW', rotation=270, labelpad=5)
ax1.set_title("Pre-Storm Elevation")
ax1.set_ylabel("barrier width (m)")
ax1.set_xlabel("barrier length (m)")
plt.gca().xaxis.tick_bottom()
xtick_max = np.shape(initial)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 10))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(initial)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 10))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)


ax2 = fig1.add_subplot(132)
mat = ax2.matshow(
    final*10,
    cmap="terrain",
    vmin=-3.0,
    vmax=6.0,
)
cbar = fig1.colorbar(mat, shrink=shrink)
cbar.set_label('m MHW', rotation=270, labelpad=5)
ax2.set_title("Post-Storm Elevation")
ax2.set_ylabel("barrier width (m)")
ax2.set_xlabel("barrier length (m)")
plt.gca().xaxis.tick_bottom()
xtick_max = np.shape(final)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 10))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(final)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 10))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)
# ax2.tick_params(left=False)
# ax2.set(yticklabels=[])
# fig2.subplots_adjust(wspace=0.3, hspace=0)

# fig3 = plt.figure()
ax3 = fig1.add_subplot(133)
mat = ax3.matshow(
    change*10,
    cmap="seismic",
    vmin=-5,
    vmax=5,
)

cbar = fig1.colorbar(mat, shrink=shrink)
cbar.set_label('meters', rotation=270, labelpad=5)
ax3.set_title("Elevation Change")
ax3.set_ylabel("barrier width (m)")
ax3.set_xlabel("barrier length (m)")
plt.gca().xaxis.tick_bottom()

# ax3.add_patch(plt.Rectangle((13,30), 24, -27, lw=3, ec="k", fc="none"))

xtick_max = np.shape(change)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 10))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(change)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 10))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)
fig1.subplots_adjust(top=1.15, wspace=0.3, hspace=0)



In [ ]:
time = 360
uw = cascade2.outwash[0]._underwater_array[1][time]
dh = cascade2.outwash[0]._downhill_array[1][time]
end = cascade2.outwash[0]._endcell_array[1][time]

plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams.update({"font.size": 15})

fig4 = plt.figure()
ax4 = fig4.add_subplot(111)
mat4 = ax4.matshow(
    uw,
    cmap="binary",
    vmin=0, vmax=3,
)
ax4.set_xlabel('barrier length (m)')
ax4.set_ylabel('barrier width (m)')
ax4.set_title("Underwater Array")
plt.gca().xaxis.tick_bottom()
xtick_max = np.shape(uw)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 10))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(uw)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 10))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)


fig4 = plt.figure()
ax4 = fig4.add_subplot(111)
mat4 = ax4.matshow(
    dh,
    cmap="binary",
    vmin=0, vmax=3,
)
ax4.set_xlabel('barrier length (m)')
ax4.set_ylabel('barrier width (m)')
# ax4.set_title("Downhill and Underwater Cells")
plt.gca().xaxis.tick_bottom()
xtick_max = np.shape(dh)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 10))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(dh)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 10))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)
plt.text(0.5, 36.5, "Light Grey = underwater cells \nDark Grey = downhill cells", fontsize="small")


fig4 = plt.figure()
ax4 = fig4.add_subplot(111)
mat4 = ax4.matshow(
    end,
    cmap="binary",
    vmin=0, vmax=3,
)
ax4.set_xlabel('barrier length (mm)')
ax4.set_ylabel('barrier width (dam)')
# ax4.set_title("Endcell Array")
plt.gca().xaxis.tick_bottom()
xtick_max = np.shape(end)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 10))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(end)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 10))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)
plt.text(0.5, 36.5, "Light Grey = underwater cells \nDark Grey = downhill cells \nBlack = start cells", fontsize="small")

In [ ]:
plt.rcParams['figure.figsize'] = (20,6)
plt.rcParams.update({"font.size": 15})

init_dis = cascade2.outwash[0]._initial_discharge[1][time] 
discharge = cascade2.outwash[0]._discharge[1][time]
row = np.shape(init_dis)[0]
col = np.shape(init_dis)[1]
for r in range(row):
    for c in range(col):
        init_dis[r,c] = init_dis[r,c]*1000/3600
        discharge[r,c] = discharge[r,c]*1000/3600


fig4 = plt.figure()
ax4 = fig4.add_subplot(121)
mat4 = ax4.matshow(
    init_dis,
    cmap="Blues",
    vmin=0, 
    vmax=300,
)
ax4.set_xlabel('barrier length (m)')
ax4.set_ylabel('barrier width (m)')
ax4.set_title("Initial Discharge")
plt.gca().xaxis.tick_bottom()
cbar = fig4.colorbar(mat4)
cbar.set_label('m$^3$/s', rotation=270, labelpad=25)
xtick_max = np.shape(init_dis)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 10))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(init_dis)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 10))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)


ax4 = fig4.add_subplot(122)
mat4 = ax4.matshow(
    discharge,
    cmap="Blues",
    vmin=0, 
    vmax=300,
)
ax4.set_xlabel('barrier length (m)')
ax4.set_ylabel('barrier width (m)')
ax4.set_title("Discharge After Flow Routing")
plt.gca().xaxis.tick_bottom()
cbar = fig4.colorbar(mat4)
cbar.set_label('m$^3$/s', rotation=270, labelpad=25)
xtick_max = np.shape(discharge)[1]  # n_cols = x
x_ticks = np.array(range(0, xtick_max, 10))
x_tick_labels = x_ticks * 10
ytick_max = np.shape(discharge)[0]  # n_rows = y
y_ticks = np.array(range(0, ytick_max, 10))
y_tick_labels = y_ticks * 10
plt.xticks(x_ticks, x_tick_labels)
plt.yticks(y_ticks, y_tick_labels)